In [48]:
import twitter
#pip install textblob
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
import re
import time
import os
import tweepy as tw
from textblob import TextBlob
import nltk

In [14]:
consumer_key="CONSUMER_KEY"
consumer_secret="CONSUMER_SECRET"
access_token_key="ACCESS_TOKEN"
access_token_secret="ACCESS_TOKEN_SECRET"
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [45]:
def clean_tweet(tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
    
def get_tweet_sentiment(tweet): 
        # create TextBlob object of passed tweet text 
        #analysis = textblob(clean_tweet(tweet)) 
        analysis = TextBlob(tweet)
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

In [46]:
#search_words=["#gulfofoman","#mariana","#war","#spill"]
search_words=["#spill","#gulfofoman","#mariana"]

for word in search_words:
    search_words = word
    date_since = "2019-07-01"

    tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(100000000000000000000000)
    tweet_data = [[tweet.user.screen_name, tweet.user.location,tweet.text,tweet.created_at,tweet.user.id] for tweet in tweets ]#if authenticate_user(tweet.user.id,10,10)]
    tweet_text = pd.DataFrame(data=tweet_data,columns=['user', "location","text","created_date","userid"])
    tweet_text["text"] =tweet_text["text"].map(clean_tweet)
    tweet_text["sentiment"] =tweet_text["text"].map(get_tweet_sentiment)
    filePath="C:\\Users\\Akshay\\Desktop\\pycode\\"+search_words+".xlsx"
    if os.path.exists(filePath):
        os.remove(filePath)
        print("File Deleted")
    else:
        print("Can not delete the file as it doesn't exists")
    tweet_text.to_excel(filePath,sheet_name=search_words)
    print("Processing completed for "+search_words)

File Deleted
Processing completed for #spill
File Deleted
Processing completed for #gulfofoman
File Deleted
Processing completed for #mariana


In [85]:
def authenticate_user(userid,friends_thres,followers_thres):
    followers=api.followers_ids(userid)
    friends=api.friends_ids(userid)
    if len(followers) > followers_thres & len(friends)>friends_thres:
        return True
    else :
        return False
    